# library

일반적인 크롤링 방식으로 검색결과를 가져오려고 하면 class 이름이라던지 id를 찾지 못해 에러가 납니다.

따라서 iframe 을 Selenium -> 창을 선택해주고 그안의 결과를 가져오는 식으로 진행해야 합니다.

In [1]:
import time # time 은 페이지를 넘어갈 때 약간의 딜레이를 줘 오류를 방지한다.

from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from pandas import DataFrame
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
import os
os.getcwd()

'C:\\Users\\Administrator\\exp-py\\proj\\suwon-yu-sns'

# 크롤링

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome()

In [36]:
sta_date = '20190603'
end_date = '20190603'

flag_save = 0
#https://cafe.naver.com/kyungmammo # 인천 아띠아모(숲, 25만 명, 대표)
driver.get("https://cafe.naver.com/kyungmammo") # 원하는 카페 주소를 입력

driver.implicitly_wait(3) # 대기시간 3초

# 원하는 검색어를 검색창에 입력
# 그리고 엔터를 입력해 검색
driver.find_element(By.NAME,'query').send_keys('적수')
driver.find_element(By.NAME,"query").send_keys(Keys.ENTER)

time.sleep(2)

driver.switch_to.frame("cafe_main")
driver.find_element(By.CSS_SELECTOR,"#currentSearchDateTop").click()
for i in range(1,11):
    driver.find_element(By.ID,"input_1_top").send_keys(Keys.BACKSPACE)
    driver.find_element(By.ID,"input_2_top").send_keys(Keys.BACKSPACE)
driver.find_element(By.ID,"input_1_top").send_keys(sta_date)
driver.find_element(By.ID,"input_2_top").send_keys(end_date)
driver.find_element(By.ID,"btn_set_top").send_keys(Keys.ENTER)
driver.find_element(By.CLASS_NAME,'btn-search-green').click()

flag_page_next = 1
     
idx_df_allpages_cases = 0
for i in range(1, 5):
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    titles = soup.select("#main-area > div:nth-child(5) > table > tbody > tr")


    list3 = []
    idx_df_tmp_cases = 0
    for title in titles:
        list = title.select_one(' td.td_article > div.board-list > div > a').text
        list2 = ''.join(list.split())
        list3.append(list2)
        idx_df_tmp_cases += 1
        idx_df_allpages_cases += 1
    df_tmp = pd.DataFrame(list3)    
           
    if i == 1:
        df_allpages = df_tmp
        print('----' + str(i) + ' 번째 페이지 -----')
        print('idx_df_allpages_cases:',idx_df_allpages_cases)
        print(df_tmp)
        
        if idx_df_tmp_cases == 15:    
            driver.find_element(By.XPATH, f'//*[@id="main-area"]/div[7]/a[{i}+1]').click() # 결과 다음 페이지로 가는 구문
    else:    
        
        # 마지막에 같은 것이 한번 더 나오게 되어 있기 때문에 무조건 마지막은 아래와 같이
        # df_allpages의 마지막 문장과 df_tmp 마지막 문장이 같으면 더이상 페이지가 없는 것
        # 넘어가지 않고, concat도 안하는 것
        if df_allpages.iloc[idx_df_allpages_cases-idx_df_tmp_cases-1].to_string() == df_tmp.iloc[idx_df_tmp_cases-1].to_string():
            idx_df_allpages_cases = idx_df_allpages_cases - idx_df_tmp_cases
            
        else:
            df_allpages = pd.concat([df_allpages, df_tmp]) # 최종이 아닌이상 concat은 해야하고, 아래는 다음페이지 넘어갈지 결정
            print('----' + str(i) + ' 번째 페이지 -----')
            print('idx_df_allpages_cases:',idx_df_allpages_cases)
            print(df_tmp)
            if idx_df_tmp_cases == 15:  
                driver.find_element(By.XPATH, f'//*[@id="main-area"]/div[7]/a[{i}+1]').click() # 결과 다음 페이지로 가는 구문
                # 다음페이지로 간 경우 iframe 은 선택 되어 있으므로 스위치를 안써줘도 된다.
                        
now = datetime.now()
if flag_save == 1:
    # 새로 만들 파일이름 지정
    outputFileName = '%s-%s-%s %s시 %s분 %s초 merging' % (now.year, now.month, now.day, now.hour, now.minute, now.second)
    df_allpages.to_excel(outputFileName+'.xlsx',sheet_name='sheet1')
    #df.to_csv(outputFileName+'.csv', mode='w')

----1 번째 페이지 -----
idx_df_allpages_cases: 15
                                0
0                    인천서구적수집회합니다!
1          [구월동]롯데힐사시는분들적수괜찮으세요??
2   [간석동]적수점점퍼지는것같아요..간석동안전할까요?ㅜㅜ
3                       적수때문에진짜ㅜㅜ
4                          주안5동적수
5                남동구도적수피해지역인가요???
6                     적수검사하고가셨어요!
7               적수문제로시청에서좀전에왔다갔네요
8                        서구지역적수관련
9                     [만수동]적수..!!
10                  미추홀구적수테스트해봤어요
11                    서구적수괜찮아졌나요?
12                     영종도적수.....
13                    적수문제조용히덮히나요
14                서구적수사태청원동의~해주세요
----2 번째 페이지 -----
idx_df_allpages_cases: 30
                          0
0   서구는적수때문에난린데미추홀구분들은어떠세요?
1             서구연희동은적수괜찮나요?
2                 가정동도적수네요‥
3        인천수도수질적수사태피해지역확인방법
4        청원부탁드려요[영종도적수피해청원]
5    혹시나해서해본적수검사...미추홀구요...
6            가정동적수저만그런가요ㅠㅠ?
7               적수..진짜짜증나네요
8      [심곡동]심곡동인데요여기도적수나옵니다
9                    적수테스트요
10       우울합니다......7일차적수..
11               저도적수확인해봤어요
12

In [32]:
idx_df_allpages_cases

7

In [24]:
df_allpages

,0
0,저희집수돗물도난리예요..ㅠㅠ
1,☆서구일대수돗물관련청원부탁드려요☆
2,부평구청천동수돗물괜찮은듯한데
3,문학동수돗물괜찮은가요?
4,학익동수돗물어떤가요?
5,부평4동수돗물괜찮은거같아요
6,수돗물
7,산곡동수돗물이요(수젛)
8,수돗물재난문자요
9,수돗물...남동구는괜찮을까요?


In [ ]:
# code fin